In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [2]:
# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
import pandas as pd
Email_Training = pd.read_excel("C:/Hackathon/arnold_clusters.xlsx",sheet=1)

In [4]:
Email_Training.head()

,Email,cluster
0,I have you buyiing 1000 NY Football Giants fro...,1
1,JohnAt your leisure can you interview this guy...,1
2,Happy New Year and best wishes. - Energy010102...,1
3,All - In preparation for another round of Tra...,4
4,GentlemenI wanted to provide you an update on ...,0


In [5]:
Email_json = Email_Training.to_json(orient = "split")

In [14]:
Email_json

'{"columns":["Email","cluster"],"index":[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131],"data":[["I have you buyiing 1000 NY Football Giants from me at $12 and paying Horowitz $8000 from which leaves a balance of  $4000. I can just get it sometime when I see you I just don\'t want to forget. ",1],["JohnAt your leisure can you interview this guy.  He is in Houston in a couple of weeks.  You may need him to resend his reume as I deleted it.  Read the email traffic below.Thanks ",1],["Happy New Year and best wishes. - Energy010102.doc ",1],["All -  In preparation for another round of Trading Track interviews for the ENA gro

In [7]:
words = []
classes = []
documents = []
ignore_words = ['?']

In [10]:
# loop through each sentence in our intents patterns
for email in Email_json['columns']:
    for pattern in Email_json['Email']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

TypeError: string indices must be integers

In [6]:
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [7]:
# remove duplicates
classes = sorted(list(set(classes)))

In [8]:
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
48 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


## This data structure won’t work with Tensorflow, we need to transform it further: from documents of words into tensors of numbers.

In [9]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [10]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [11]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

## Our data is shuffled. Tensorflow will take some of this and use it as test data to gauge accuracy for a newly fitted model.

In [12]:
# Building our model
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

In [13]:
# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('C:/ChatBot/Tensorflow/model.tflearn')

Training Step: 3999  | total loss: 0.14488 | time: 0.020s
| Adam | epoch: 1000 | loss: 0.14488 - acc: 0.9988 -- iter: 24/27
Training Step: 4000  | total loss: 0.13696 | time: 0.020s
| Adam | epoch: 1000 | loss: 0.13696 - acc: 0.9989 -- iter: 27/27
--
INFO:tensorflow:C:/ChatBot/Tensorflow/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [16]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [17]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [18]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0]
['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']


In [19]:
print(model.predict([p]))

[[4.629927730093186e-07, 0.0004036641039419919, 0.12859593331813812, 2.529509117721318e-07, 0.8209863901138306, 0.010348940268158913, 3.8250490774771606e-08, 3.1558582122670487e-05, 0.03963283821940422]]


## We’ll save (‘pickle’) our model and documents for future use

In [20]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "C:/ChatBot/Tensorflow/training_data", "wb" ) )